In [1]:
!pip -q install transformers
!pip -q install sentencepiece

In [2]:
!pip -q install datasets

In [3]:
import tensorflow as tf
import transformers
import datasets
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re

print(tf.__version__, transformers.__version__) # 
datasets.__version__

2.4.0 4.1.1


'1.2.0'

In [4]:
from datasets import load_dataset

scb_then_ds = load_dataset('scb_mt_enth_2020','then') # download-size 135MB
scb_enth_ds = load_dataset('scb_mt_enth_2020','enth') # download-size 135MB
thaiqa_ds = load_dataset('thaiqa_squad')
# thaisum_ds = load_dataset('thaisum') # download-size 650MB
# prachatai_ds = load_dataset('prachathai67k') # download-size 254MB
wongnai_ds = load_dataset('wongnai_reviews')
wisesight_ds = load_dataset('wisesight_sentiment')

Downloading: 4.08kB [00:00, 1.03MB/s]                   
Downloading: 100%|██████████| 138M/138M [05:12<00:00, 442kB/s]
Dataset scb_mt_enth2020 downloaded and prepared to C:\Users\user\.cache\huggingface\datasets\scb_mt_enth2020\then\1.0.0\6ba9d692e0ef81f29bd3ebff31ca1a92ed8338388da4f0c12f2c8c28cbc70b9a. Subsequent calls will reuse this data.
Dataset scb_mt_enth2020 downloaded and prepared to C:\Users\user\.cache\huggingface\datasets\scb_mt_enth2020\enth\1.0.0\6ba9d692e0ef81f29bd3ebff31ca1a92ed8338388da4f0c12f2c8c28cbc70b9a. Subsequent calls will reuse this data.
Downloading: 4.34kB [00:00, 1.09MB/s]                   
Downloading: 100%|██████████| 10.0M/10.0M [00:10<00:00, 956kB/s]
Dataset thaiqa_squad downloaded and prepared to C:\Users\user\.cache\huggingface\datasets\thaiqa_squad\thaiqa_squad\1.0.0\7b7e2b91d89dc82fb5f3daebad1f7760eff94f7df63eaee9c8de1dd700c090b9. Subsequent calls will reuse this data.
Downloading: 2.62kB [00:00, 378kB/s]                    
Using custom data config

In [5]:
print(thaiqa_ds)
print(scb_then_ds)

DatasetDict({
    train: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 74
    })
})
DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 801402
    })
    validation: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 100173
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 100177
    })
})


In [6]:
scb_enth_ds['train'][0], scb_then_ds['train'][0]

({'subdataset': 'aqdf_corpus',
  'translation': {'en': 'FAR LEFT: Indonesian National Police Chief Tito Karnavian, from left, Philippine National Police Chief Ronald Dela Rosa and Royal Malaysian Police Inspector General Khalid Abu Bakar link arms before the Trilateral Security Meeting in Pasay city, southeast of Manila, Philippines, in June 2017. [THE ASSOCIATED PRESS]',
   'th': '(ซ้ายสุด) นายติโต คาร์นาเวียน ผู้บัญชาการตํารวจแห่งชาติอินโดนีเซีย (จากซ้าย) นายโรนัลด์ เดลา โรซา ผู้บัญชาการตํารวจแห่งชาติฟิลิปปินส์ และนายคาลิด อาบู บาการ์ ผู้บัญชาการตํารวจแห่งชาติมาเลเซีย ไขว้แขนกันก่อนเริ่มการประชุมความมั่นคงไตรภาคีในเมืองปาเซย์ ซึ่งอยู่ทางตะวันออกเฉียงใต้ของกรุงมะนิลา ประเทศฟิลิปปินส์ ในเดือนมิถุนายน พ.ศ. 2560 ดิแอสโซซิเอทเต็ด เพรส'}},
 {'subdataset': 'aqdf_corpus',
  'translation': {'en': 'FAR LEFT: Indonesian National Police Chief Tito Karnavian, from left, Philippine National Police Chief Ronald Dela Rosa and Royal Malaysian Police Inspector General Khalid Abu Bakar link arms before